# Inference: Adaptive Covariance: localised MCMC

This example shows you how to perform Bayesian inference on a time series, using a variant of [Adaptive Covariance MCMC](http://pints.readthedocs.io/en/latest/mcmc_samplers/adaptive_covariance_mcmc_remi.html) detailed in Algorithm 7 of [1], which involves using a vector of possible proposal distributions.

[1] A tutorial on adaptive MCMC
    Christophe Andrieu and Johannes Thoms, Statistical Computing,
    2008, 18: 343-373

It follows on from the [first sampling example](./sampling-first-example.ipynb).

In [1]:
import os
os.chdir('C:/Users/bclamber/Desktop/AllPints/pints4/pints')
from __future__ import print_function
import pints
import pints.toy as toy
import pints.plot
import numpy as np
import matplotlib.pyplot as plt

# Load a forward model
model = toy.LogisticModel()

# Create some toy data
real_parameters = [0.015, 500]
times = np.linspace(0, 1000, 1000)
org_values = model.simulate(real_parameters, times)

# Add noise
noise = 10
values = org_values + np.random.normal(0, noise, org_values.shape)
real_parameters = np.array(real_parameters + [noise])

# Get properties of the noise sample
noise_sample_mean = np.mean(values - org_values)
noise_sample_std = np.std(values - org_values)

# Create an object with links to the model and time series
problem = pints.SingleOutputProblem(model, times, values)

# Create a log-likelihood function (adds an extra parameter!)
log_likelihood = pints.UnknownNoiseLogLikelihood(problem)

# Create a uniform prior over both the parameters and the new noise variable
log_prior = pints.UniformLogPrior(
    [0.01, 400, noise*0.1],
    [0.02, 600, noise*100]
)

# Create a posterior log-likelihood (log(likelihood * prior))
log_posterior = pints.LogPosterior(log_likelihood, log_prior)

# Choose starting points for 3 mcmc chains
xs = [
    real_parameters * 1.1,
    real_parameters * 0.9,
    real_parameters * 1.15,
]

# Create mcmc routine with four chains
mcmc = pints.MCMCSampling(log_posterior, 3, xs, method=pints.AdaptiveCovarianceLocalisedMCMC)

# Add stopping criterion
mcmc.set_max_iterations(4000)

# Start adapting after 1000 iterations
mcmc.set_initial_phase_iterations(1000)

# Disable logging mode
mcmc.set_log_to_screen(False)

# Run!
print('Running...')
chains = mcmc.run()
print('Done!')

# Show traces and histograms
pints.plot.trace(chains)

# Discard warm up
chains = chains[:, 2000:, :]

# Check convergence using rhat criterion
print('R-hat:')
print(pints.rhat_all_params(chains))

# Look at distribution in chain 0
pints.plot.pairwise(chains[0], kde=True)

# Show graphs
plt.show()

Running...
[array([[  1.65000000e-04,   0.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   5.50000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   1.10000000e-01]]), array([[  1.65000000e-04,   0.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   5.50000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   1.10000000e-01]]), array([[  1.65000000e-04,   0.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   5.50000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   1.10000000e-01]])]
[array([[  1.35000000e-04,   0.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   4.50000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   9.00000000e-02]]), array([[  1.35000000e-04,   0.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   4.50000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   9.00000000e-02]]), array([[  1.35000000e-04,   0.00

C:\Users\bclamber\Desktop\AllPints\pints4\pints\pints\_mcmc\_adaptive_covariance.py:160: RuntimeWarning: overflow encountered in exp
  self._alpha = np.minimum(1, np.exp(r + self._ratio_q()))


[array([[  1.35000000e-04,   0.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   4.50000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   9.00000000e-02]]), array([[  1.35000000e-04,   0.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   4.50000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   9.00000000e-02]]), array([[  1.35000000e-04,   0.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   4.50000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   9.00000000e-02]])]
[array([[  1.72500000e-04,   0.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   5.75000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   1.15000000e-01]]), array([[  1.72500000e-04,   0.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   5.75000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   1.15000000e-01]]), array([[  1.72500000e-04,   0.00000000e+00,

[array([[  1.65000000e-04,   0.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   5.50000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   1.10000000e-01]]), array([[  1.65000000e-04,   0.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   5.50000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   1.10000000e-01]]), array([[  1.65000000e-04,   0.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   5.50000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   1.10000000e-01]])]
[array([[  1.35000000e-04,   0.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   4.50000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   9.00000000e-02]]), array([[  1.35000000e-04,   0.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   4.50000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   9.00000000e-02]]), array([[  1.35000000e-04,   0.00000000e+00,

[array([[  1.35000000e-04,   0.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   4.50000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   9.00000000e-02]]), array([[  1.35000000e-04,   0.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   4.50000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   9.00000000e-02]]), array([[  1.35000000e-04,   0.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   4.50000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   9.00000000e-02]])]
[array([[  1.72500000e-04,   0.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   5.75000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   1.15000000e-01]]), array([[  1.72500000e-04,   0.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   5.75000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   1.15000000e-01]]), array([[  1.72500000e-04,   0.00000000e+00,

...

       [  5.54254913e+231,   9.23702721e+234,   1.63970026e+235]])]
[array([[  3.55330569e+232,   1.09378256e+233,  -1.79707875e+232],
       [  1.09378256e+233,   1.28264057e+237,  -1.67767422e+235],
       [ -1.79707875e+232,  -1.67767422e+235,   2.63670056e+235]]), array([[  3.55330569e+232,   1.09378256e+233,  -1.79707875e+232],
       [  1.09378256e+233,   1.28264057e+237,  -1.67767422e+235],
       [ -1.79707875e+232,  -1.67767422e+235,   2.63670056e+235]]), array([[  3.55330569e+232,   1.09378256e+233,  -1.79707875e+232],
       [  1.09378256e+233,   1.28264057e+237,  -1.67767422e+235],
       [ -1.79707875e+232,  -1.67767422e+235,   2.63670056e+235]])]
[array([[  2.76542295e+233,   1.19443550e+234,  -2.99103280e+232],
       [  1.19443550e+234,   1.29187323e+238,  -9.29902570e+235],
       [ -2.99103280e+232,  -9.29902570e+235,   1.86434010e+236]]), array([[  2.76542295e+233,   1.19443550e+234,  -2.99103280e+232],
       [  1.19443550e+234,   1.29187323e+238,  -9.29902570e+235]

[array([[  1.04474725e+256,   4.51244973e+256,  -1.12998024e+255],
       [  4.51244973e+256,   4.88055906e+260,  -3.51307258e+258],
       [ -1.12998024e+255,  -3.51307258e+258,   7.04327778e+258]]), array([[  1.04474725e+256,   4.51244973e+256,  -1.12998024e+255],
       [  4.51244973e+256,   4.88055906e+260,  -3.51307258e+258],
       [ -1.12998024e+255,  -3.51307258e+258,   7.04327778e+258]]), array([[  1.04474725e+256,   4.51244973e+256,  -1.12998024e+255],
       [  4.51244973e+256,   4.88055906e+260,  -3.51307258e+258],
       [ -1.12998024e+255,  -3.51307258e+258,   7.04327778e+258]])]
[array([[  7.12371599e+255,   2.17271856e+256,   1.67513269e+255],
       [  2.17271856e+256,   2.73232432e+260,   2.79172017e+258],
       [  1.67513269e+255,   2.79172017e+258,   4.95568994e+258]]), array([[  7.12371599e+255,   2.17271856e+256,   1.67513269e+255],
       [  2.17271856e+256,   2.73232432e+260,   2.79172017e+258],
       [  1.67513269e+255,   2.79172017e+258,   4.95568994e+258]])

       [ -2.73212459e+279,  -8.49408831e+282,   1.70296007e+283]])]
[array([[  1.72240884e+280,   5.25331114e+280,   4.05022231e+279],
       [  5.25331114e+280,   6.60635482e+284,   6.74996520e+282],
       [  4.05022231e+279,   6.74996520e+282,   1.19821230e+283]]), array([[  1.72240884e+280,   5.25331114e+280,   4.05022231e+279],
       [  5.25331114e+280,   6.60635482e+284,   6.74996520e+282],
       [  4.05022231e+279,   6.74996520e+282,   1.19821230e+283]]), array([[  1.72240884e+280,   5.25331114e+280,   4.05022231e+279],
       [  5.25331114e+280,   6.60635482e+284,   6.74996520e+282],
       [  4.05022231e+279,   6.74996520e+282,   1.19821230e+283]])]
[array([[  2.59658104e+280,   7.99282499e+280,  -1.31321677e+280],
       [  7.99282499e+280,   9.37290645e+284,  -1.22596181e+283],
       [ -1.31321677e+280,  -1.22596181e+283,   1.92677109e+283]]), array([[  2.59658104e+280,   7.99282499e+280,  -1.31321677e+280],
       [  7.99282499e+280,   9.37290645e+284,  -1.22596181e+283]

C:\Users\bclamber\Desktop\AllPints\pints4\pints\pints\_mcmc\_adaptive_covariance_localised.py:192: RuntimeWarning: overflow encountered in add
  self._gamma * np.exp(self._log_q_l[i]) * np.dot(dsigm, dsigm.T))


ValueError: array must not contain infs or NaNs

In [ ]:
from sklearn import mixture
import numpy as np

In [ ]:
n_samples = 500
C = np.array([[0., -0.1], [1.7, .4]])
X = np.r_[np.dot(np.random.randn(n_samples, 2), C),
          .7 * np.random.randn(n_samples, 2) + np.array([-6, 3])]


In [ ]:
gmm = mixture.GaussianMixture(n_components=4, covariance_type='full').fit(X)

In [ ]:
gmm.aic(X)

In [ ]:
gmm.means_

In [ ]:
gmm.weights_

In [ ]:
[4 for i in range(0,5)]

In [ ]:
from scipy.stats import multivariate_normal
mu = [[1,1], [1,2]]
sigma = [[[1,0],[0,1]],[[2,0],[0,1]]]
i = 0
lambda x: multivariate_normal.logpdf(x, mu[i], self._sigma[i])

In [ ]:
from scipy.stats import multivariate_normal
lambda x: multivariate_normal.logpdf(x, mu[1], self._sigma[1])

In [ ]:
q_l = [lambda x: multivariate_normal.logpdf(x, mu[i], self._sigma[i]) for i in range(2)]

In [ ]:
np.sum([1,2,3])

In [ ]:
from scipy.misc import logsumexp

In [ ]:
logsumexp([1,2,3])

In [ ]:
np.array([1,2,3])-1

In [ ]:
np.random.choice(3, 1, [ 0.33333333,  0.33333333,  0.33333333])[0]

In [ ]:
np.random.choice(3, 1, np.array([ 0.33333333,  0.33333333,  0.33333333]))[0]

In [ ]:
a=np.array([ 0.33333333,  0.33333333,  0.33333333])
a.tolist()

In [ ]:
a